In [ ]:
# !pip install -r "RAG_requirement.txt"

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   --- ------------------------------------ 1.0/10.8 MB 4.2 MB/s eta 0:00:03
   ------------ --------------------------- 3.4/10.8 MB 8.1 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/10.8 MB 8.4 MB/s eta 0:00:01
   ----------------------------- ---------- 7.9/10.8 MB 9.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/10.8 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 10.8/10.8 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/558.7 kB ? eta -:--:--
   --------------------------------------- 558.7/558.7 kB 18.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 13.1 MB/s eta 0:00:00
   ---------------------

In [ ]:
import ollama

p = "1+1= เท่าไหร่" #พรอม
res = ollama.chat(model= "llama3.2", messages=[
    {"role": "user", "content": p}
])
res["message"] #show output

# ทดลอง llama3.2 LLM

Message(role='assistant', content='1+1=2', thinking=None, images=None, tool_calls=None)

In [ ]:
import psycopg2

conn = psycopg2.connect(
    dbname="fitderdb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
) #connect to db

cur = conn.cursor()

# cur.execute("CREATE EXTENSION vector;") # สร้าง table ไว้เก็บ vector (ทำครั้งเดียว)

cur.execute("""CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            content TEXT,
            embedding vector(1024)
            )""") # setting database

conn.commit()
cur.close()
conn.close()

In [ ]:
from sentence_transformers import SentenceTransformer

conn = psycopg2.connect(
    dbname="fitderdb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

embed = SentenceTransformer("BAAI/bge-m3") # ตัวแปลง text to vec

def add_doc(text): #ใส่ข้อมูลเข้า db
    em = embed.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, em) VALUES (%s, %s)", (text, em))
    conn.commit

documents = [] #ใส่ part data

for doc in documents:
    add_doc(doc)

cur.close()
conn.close()


c:\Users\NBODT\Documents\GitHub\FitDer-Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NBODT\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[-0.04088950902223587,
 0.028241198509931564,
 -0.05546334758400917,
 -0.01174737699329853,
 -0.02095704711973667,
 -0.04442906379699707,
 -0.01374029740691185,
 -0.04919154942035675,
 0.03366753086447716,
 -0.005677435547113419,
 0.0013314135139808059,
 0.01847676932811737,
 -0.006686747539788485,
 -0.019070830196142197,
 0.013093667104840279,
 -0.037040047347545624,
 0.007444863207638264,
 -0.036237914115190506,
 -0.0005795616307295859,
 -0.046320702880620956,
 -0.020222291350364685,
 -0.0022222823463380337,
 0.03431783244013786,
 0.007922179065644741,
 9.244744433090091e-05,
 0.029368503019213676,
 -0.021959176287055016,
 0.01269594207406044,
 0.03352892026305199,
 -0.01190248504281044,
 0.028024986386299133,
 0.05831966549158096,
 -0.010952443815767765,
 -0.027744978666305542,
 -0.029990030452609062,
 -0.07778218388557434,
 0.007983949966728687,
 -0.024082407355308533,
 -0.06830699741840363,
 0.032149165868759155,
 0.04224536567926407,
 0.003090391866862774,
 0.018780017271637917,


In [ ]:
def query_post(q_text, k=3): # หาความเหมือนของ vector
    query_em = embed.encode(q_text).tolist()
    conn = psycopg2.connect(
        dbname="fitderdb",
        user="admin",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()
    query_em_str = "[" + ", ".join(map(str, query_em)) + "]" 

    sql_query = """
        SELECT content, embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
    """

    cur.execute(sql_query, (query_em_str, k))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

x = query_post("text")
x

In [ ]:
import ollama

def Gen_Res(query_text): # ยัด LLM
    ret_docs = query_post(query_text)
    context = "\n".join([doc[0] for doc in ret_docs])
    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query_text}"

    response = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content": "You are a trainer fitness."},
        {"role": "user", "content": prompt}
    ])
    return response["message"]["context"]

Gen_Res("test text")